In [ ]:
%matplotlib inline
import numpy as np
import lib.io.stan
import lib.syn_data
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)
nn = 5
SC = lib.syn_data.gen_con(nn)
# SC = np.zeros([nn,nn])
# SC[4,:] = 0.0
# SC[4,2] = 1.0
# SC[1,4] = 0.05
im = plt.imshow(SC)
plt.xticks(np.arange(0,nn),map(str,np.arange(1,nn+1)))
plt.yticks(np.arange(0,nn),map(str,np.arange(1,nn+1)))
plt.colorbar(fraction=0.046, pad=0.04)
plt.gca().set_title("Structural Connectivity",fontweight="bold")

np.savez('results/exp2/network.npz', SC=SC)

In [ ]:
I1 = 3.1
tau0 = 30.0
sigma = 0.0001
k = -2.0
Ic = np.zeros(nn)

x0 = -2.5*np.ones(nn)
x0[2] = -1.8
x0[4] = -2.2
time_scale = 1.0
time_step = 0.1
nt = 500*int(1/time_step)
sigma_xobs = 0.1
sigma_zobs = 0.1

x_init = -0.5*np.ones(nn)
z_init = 1.5*np.ones(nn)
z_eta = np.random.normal(0.0, 1.0, [nt-1, nn])

data = {'nn':nn, 'nt':nt, 'I1':I1, 'tau0':tau0, 'Ic':Ic, 'SC':SC, 
        'sigma':sigma, 'k':k, 'sigma_xobs':sigma_xobs,'sigma_zobs':sigma_zobs,        
        'x0':x0, 'time_scale':time_scale, 'time_step':time_step,
        'x_init':x_init, 'z_init':z_init, 'z_eta': z_eta}

lib.io.stan.rdump('results/exp2/sim_data.R',data)

stan_fname = './vep-srcfit-sim'
lib.io.stan.create_process(['bash','/home/anirudhnihalani/scripts/stancompile.sh', stan_fname],block=True)
sim_cmd = stan_fname +' sample algorithm=fixed_param num_samples=1 num_warmup=0 data \
file=results/exp2/sim_data.R output file=results/exp2/sim_out.csv'
lib.io.stan.create_process(sim_cmd.split(),block=True)

In [ ]:
t = lib.io.stan.parse_csv('results/exp2/sim_out.csv')
x = t['x'][0]
z = t['z'][0]
xobs = t['xobs'][0]
zobs = t['zobs'][0]


plt.figure(figsize=[15,4])
plt.subplot(121)
for i in range(nn):
    plt.plot(x.T[:,i]/4.0 + i,label=f'node{i+1}',color='black')
plt.yticks(np.mean(x.T/4.0 + np.arange(nn),axis=0), map(lambda x: f'node {x}',np.arange(1,nn+1)))
plt.xlabel('time')
plt.gca().set_title('x',fontsize=15.0)
plt.subplot(122)
for i in range(nn):
    plt.plot(z.T[:,i]/4.0 + i,label=f'node{i}',color='black')
plt.yticks(np.mean(z.T/4.0 + np.arange(nn),axis=0), map(lambda x: f'node {x}',np.arange(1,nn+1)))
plt.xlabel('time')
plt.gca().set_title('z',fontsize=15.0)
plt.suptitle('Source Signals')


plt.figure(figsize=[15,4])
plt.subplot(121)
for i in range(nn):
    plt.plot(xobs.T[:,i]/4.0 + i,label=f'node{i+1}',color='black')
plt.yticks(np.mean(xobs.T/4.0 + np.arange(nn),axis=0), map(lambda x: f'node {x}',np.arange(1,nn+1)))
plt.xlabel('time')
plt.gca().set_title('x',fontsize=15.0)
plt.subplot(122)
for i in range(nn):
    plt.plot(zobs.T[:,i]/4.0 + i,label=f'node{i}',color='black')
plt.yticks(np.mean(zobs.T/4.0 + np.arange(nn),axis=0), map(lambda x: f'node {x}',np.arange(1,nn+1)))
plt.xlabel('time')
plt.gca().set_title('z',fontsize=15.0)
plt.suptitle('Observed source signals')

In [ ]:
plt.figure(figsize=[15,3])
for i in range(nn):
    plt.subplot(1,5,i+1)
    plt.plot(x[i,:],'k')
    plt.xlabel('time')
    plt.ylabel('x',fontsize=15.0)
    plt.ylim([-3.0,1.0])
    plt.gca().set_title(f'node {i+1}')
    
    
plt.tight_layout()
plt.figure(figsize=[15,3])
for i in range(nn):
    plt.subplot(1,5,i+1)
    plt.plot(z[i,:],'k')
    plt.xlabel('time')
    plt.ylabel('z',fontsize=15.0)
    plt.ylim([2.0,5.0])
    plt.gca().set_title(f'node {i+1}')
plt.tight_layout()

In [ ]:
data = {'nn':nn, 'nt':nt, 'I1':I1, 'tau0':tau0, 'Ic':Ic, 'SC':SC, 
        'sigma':sigma, 'k':k, 'sigma_xobs':sigma_xobs,        
        'x0':x0, 'time_scale':time_scale, 'time_step':time_step,
        'x_init':x_init, 'z_init':z_init, 'z_eta': z_eta, 'xobs': xobs.T}

lib.io.stan.rdump('results/exp2/fit_data.R',data)

stan_fname = './vep-srcfit'
lib.io.stan.create_process(['bash','/home/anirudhnihalani/scripts/stancompile.sh', stan_fname],block=True)

results_dir = 'results/exp2'
nchains = 8
with open('vep-srcfit.sh','r') as fd:
    slurm_script = fd.read().format(results_dir,nchains)
with open('tmp/vep-srcfit.sh','w') as fd:
    fd.write(slurm_script)
lib.io.stan.create_process(['sbatch','tmp/vep-srcfit.sh'],block=False)
